##대일항쟁기 강제동원 희생자 지도 시각화
### 과제 요청 부서 : 과거사관련업무지원단 - 대일항쟁기강제동원피해지원과
### 과제 수행 기간 : 2024.03.~2024.08.
### 과제 담당 : 데이터정보화담당관 청년인턴 이선희
### e-mail : allmysullem223@gmail.com
---
<코드 개요>   
지도 시각화를 위하여 위도 경도 필요   
희생자 목록에 존재하고 있는 '동원지역명 - 수정' 열을 이용하여 위도 경도 추출 수행하기 위한 코드

---
<과정>
1. 비어있거나 대치 시켜야 하는 지역명 변경
2.  googletrans를 사용하요 한국어 지역명을 이용하여 다양한 언어의 지역명으로 번역 수행 후 새로운 열 생성
3. googlemaps를 이용하여 위도 경도 추출
4. 데이터 확정

---
<코드 실행시 주의사항>   
- API보안문제로 인하여 업무용 인터넷망 PC에서는 수행 불가능
- 코드는 개인 노트북을 이용하여 google colab을 사용하여 수행해야함
- key값은 개인 key값을 발급 받아 사용
- 많은 동원지역명을 추출시도할 경우에는 사용할 수 있지만 소수의 지역의 위도 경도를 추출할 경우에는 엑셀 데이터에서 직접 변경하는 것이 수월함
- 특히 시대가 변화하며 현재의 지역명과 다른 지역명들이 다수 존재 + 지역명이 아닌 작은 마음을 명칭도 있기 때문에 위도 경도 추출이 쉽지 않음
- 제대로 추출되지 않는 경우 구글 지도를 사용하여 직접 검색 or 과거사관련업무 지원단 선생님들께 자료 요청 드리는 것이 빠름

# 0. google drive
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. 지역명 엑셀 데이터 불러오기
----

In [ ]:
# 판다스 불러오기
import pandas as pd

In [ ]:
# 지역명.xlsx 데이터 불러오기
local = pd.read_excel('엑셀 경로 입력')

In [ ]:
local.head()

# 2. 지역명을 다른 언어로 번역
---
- 한국어로 된 지역명에서 위도, 경도 추출하는 것이 어려울 수 있음
- 지역명을 번역하기 위해  googletrans
- 파파고 유료로 전환되어 사용 불가능

1) googletrans 라이브러리 설치   
- pip install googletrans를 하면 동작하지 않는 버전을 설치하므로 버전지정이 필수  
- 잘못 설치했다면 pip uninstall googletrans로 지워주고 다시 설치   

In [ ]:
!pip install googletrans==4.0.0-rc.1

2) Translator import 시키기  
- translator.translate( 번역할 문장(string) ,src='번역할 문장의  언어' ,dest='변경할 언어' )

In [ ]:
from googletrans import Translator
translator = Translator()

3) 여러 언어 한번에 변환하기
- 한국어를 영어, 일본어, 중국어 순으로 번역
- 영어 : 'en'
- 일본어 : 'ja'
- 중국어 : 'zh-CN'

In [ ]:
# 번역할 언어 리스트 생성
# 필요시 다양한 언어 추가하여 list로 생성
language_list = ['en', 'ja','zh-CN']

In [ ]:
# 번역할 언어 리스트가 담길 열을 생성하기 위해 변수 선언
new_column = '동원지역명 수정'

# 번역된 결과를 저장할 컬럼 이름 리스트 생성
# language_list의 값을 for문을 이용해 하나씩 가져와 f_string을 사용해 새로운 컬럼명 생성 후 translated_columns에 반환
translated_columns = [f'{new_column}_{i}' for i in language_list]

In [ ]:
# 생성된 list
translated_columns

함수 : trans_languages_list
1. translator에 Translaor()라는 객체 생성
2. trans_text라는 번역함수 정의
3. 각 언어에 대해 번역된 결과를 저장할 열 추가

In [ ]:
def trans_languages_list(df, src_column, dest_columns, languages):

    # Translator 객체 생성
    translator = Translator()

    # 번역 함수 정의
    def trans_text(text, dest_lang):
        return translator.translate(text, dest=dest_lang).text

    # 각 언어에 대해 번역된 결과를 저장할 열 추가
    for dest_column, lang in zip(dest_columns, languages):
        df[dest_column] = df[src_column].apply(trans_text, args=(lang,))

    return df


# 함수 호출
# trans_languages_list(데이터프레임명, 번역할 컬럼, 번역된 내용이 담길 컬럼, 번역할 언어 리스트)
trans_languages_list(local, '동원지역명', translated_columns, language_list)


In [ ]:
local.rename(columns={'동원지역명': '동원지역명_한국어',
                      '동원지역명 수정_en': '동원지역명_영어',
                      '동원지역명 수정_ja': '동원지역명_일본어',
                      '동원지역명 수정_zh-CN': '동원지역명_중국어'}, inplace=True)

In [ ]:
local.head(3)

*주의할 점*
- 번역할 때 지역명이 아닌 다른 동의어로 번역될 가능성 농후함으로 확인 필요

# 3. 위도, 경도 추출하기
---
- 열의 이름은 [위도(y)], [경도(x)]를 사용
- 위도(y) : 위도(latitude)값은 y값
- 경도(x) : 경도(longitude)값은 x값

In [ ]:
# Google Maps를 사용하기 위한 패키지
!pip install -U googlemaps

In [ ]:
import googlemaps

In [ ]:
# Google Maps API 키
key = 'googlemaps 발급받은 키 입력'

In [ ]:
# Google Maps API 클라이언트 생성
gmaps = googlemaps.Client(key=key)

In [ ]:
# 위도, 경도 담을 공간 생성
local['위도'] =''
local['경도'] =''

In [ ]:
local

In [ ]:
# 사용 x
# # 한국어로 위경도 추출
# for index, row in local.iterrows():
#     geocode_result = gmaps.geocode(row['동원지역명_한국어'])
#     if geocode_result:
#         location = geocode_result[0]['geometry']['location']
#         local.at[index, '위도'] = location['lat']
#         local.at[index, '경도'] = location['lng']

1) 한국 예외 처리를 하지 않은 경우

In [ ]:
# 사용 x
# # Geocoding 함수 정의
# def get_geocode(row, columns_to_try):
#     for column in columns_to_try:
#         geocode_result = gmaps.geocode(row[column])
#         if geocode_result:
#             location = geocode_result[0]['geometry']['location']
#             return location['lat'], location['lng']
#     return None, None

# # 위도 경도 추출
# for index, row in local.iterrows():
#     # 섬 이름, 섬 영어 이름, 섬 일본어 이름 순으로 시도
#     lat, lng = get_geocode(row, ['동원지역명_한국어','동원지역명_영어','동원지역명_일본어','동원지역명_중국어'])
#     local.at[index, '위도'] = lat
#     local.at[index, '경도'] = lng


In [ ]:
local

2) 한국 예외 처리를 한 경우

In [ ]:
# 사용 x
# # Geocoding 함수 정의
# def get_geocode(row, columns_to_try):
#     for column in columns_to_try:
#         geocode_result = gmaps.geocode(row[column])
#         if geocode_result:
#             location = geocode_result[0]['geometry']['location']

#             # 한국 범위 안에 위도와 경도가 들어오는 경우
#             if 33 <= location['lat'] <= 38 and 124 <= location['lng'] <= 132:
#                 # 다음 열로 이동하여 번역을 시도
#                 next_column_index = columns_to_try.index(column) + 1
#                 if next_column_index < len(columns_to_try):
#                     next_column = columns_to_try[next_column_index]
#                     # 다음 열의 동원지역명_영어로 이동하여 번역을 시도
#                     return get_geocode(row, [next_column])
#                 else:
#                     # 번역할 열이 더 이상 없는 경우 None 반환
#                     return None, None
#             return location['lat'], location['lng']
#     return None, None

# # 위도 경도 추출
# for index, row in local.iterrows():
#     # 섬 이름, 섬 영어 이름, 섬 일본어 이름 순으로 시도
#     lat, lng = get_geocode(row, ['동원지역명_한국어', '동원지역명_영어', '동원지역명_일본어','동원지역명_중국어'])
#     local.at[index, '위도'] = lat
#     local.at[index, '경도'] = lng


### 한국 예외처리가 필요한 이유
googlemaps를 이용하여 위도 경도 추출시 가장 먼저 추출하는 언어는 한국어   
한국에 위치하고 있는 상호명과 추출하려는 지역명이 일치할 경우 원하지 않은 곳이 추출될 가능성 높음
그렇기 때문에 한국의 범위((33,125)에서(37,130)) 내에 위치할 경우 다음 언어로 재검색 시도할 수 있는 코드 필요

In [ ]:
# 가장 효율적인 코드
def get_geocode(row, columns_to_try, current_index=0):

    if current_index >= len(columns_to_try):
        # 현재 인덱스가 열 목록을 벗어난 경우 None 반환
        return None, None

    column = columns_to_try[current_index]
    geocode_result = gmaps.geocode(row[column])

    if geocode_result:
        location = geocode_result[0]['geometry']['location']

        # 한국 범위 안에 위도와 경도가 들어오는 경우
        if 33 <= location['lat'] <= 37  and 125 <= location['lng'] <= 130:
            # 다음 열로 이동하여 번역을 시도
            next_index = current_index + 1
            return get_geocode(row, columns_to_try, next_index)
        else:
            return location['lat'], location['lng']
    else:
        # 결과가 없는 경우 현재 열에서 추출을 시도하지 않고 다음 열로 이동하여 재시도
        next_index = current_index + 1
        return get_geocode(row, columns_to_try, next_index)


# 위도 경도 추출
for index, row in local.iterrows():
    # 섬 이름, 섬 영어 이름, 섬 일본어 이름 순으로 시도
    lat, lng = get_geocode(row, ['동원지역명_한국어', '동원지역명_영어', '동원지역명_일본어', '동원지역명_중국어'])

    if lat is not None and lng is not None:
        local.at[index, '위도'] = lat
        local.at[index, '경도'] = lng
    else:
        # 위도와 경도가 없는 경우 None으로 업데이트
        local.at[index, '위도'] = None
        local.at[index, '경도'] = None


# 4. 위도, 경도 folium을 사용하여 지도 출력
---

In [ ]:
local

In [ ]:
import folium

In [ ]:
# 지도 초기 위치 설정
m = folium.Map(location=[0, 0], zoom_start=2)

In [ ]:
# 국가별 좌표를 지도에 표시
# iterrows()는 DataFrame의 각 행을 반복적으로 반환하는 함수
# iterrows() 메서드는 반복 가능한 튜플 형태로 각 행의 인덱스와 데이터를 반환
for index, row in local.iterrows():
    country_name = row['동원지역명_한국어']
    latitude = row['위도']
    longitude = row['경도']

    #좌표가 NAN일 경우에는 표시 x
    if pd.notnull(latitude) and pd.notnull(longitude):
        folium.Marker([latitude, longitude], tooltip=country_name).add_to(m)

# 지도를 HTML 파일로 저장
m.save("map_with_coordinates.html")

In [ ]:
# 지도 출력하기
m

# 5. 위도, 경도 데이터 엑셀로 저장
엑셀 데이터로 저장할 경우에는 번역된 동원지역명 삭제

In [ ]:
drop_list = ['동원지역명_영어', '동원지역명_일본어','동원지역명_중국어']
local = local.drop(drop_list, axis=1)

In [ ]:
local.rename(columns={'동원지역명_한국어':'동원지역명 수정'}, inplace=True)

In [ ]:
local

In [ ]:
# 업데이트 확인을 위한 파일명 날짜 수정 필수
local.to_excel('/content/drive/MyDrive/QGIS/5월09일/동원지역명 확인용 (날짜).xlsx.xlsx', index=False)